In [1]:
!pip install faker

In [1]:
import random
import pandas as pd
import numpy as np

from faker import Faker

from bs4 import BeautifulSoup
import requests

In [2]:
fake = Faker('pl_PL')

In [3]:
# sprawdzanie biblioteki
adres = fake.address()
adres

'al. Modrzewiowa 29\n34-941 Zgierz'

In [4]:
ulica = fake.street_address()
kod_pocztowy = fake.postcode()
miasto = fake.city()
print(f"{ulica} {kod_pocztowy} {miasto}")

al. Paderewskiego 73 89-851 Zgierz


In [6]:
# scrappowanie danych ze strony internetowej
url = 'https://www.kody-pocztowe.dokladnie.com/okreg5.php'
response = requests.get(url).content
soup = BeautifulSoup(response, 'html.parser')
scrapped = soup.select('#page-table td:nth-child(1) , #page-table td:nth-child(3)')

In [7]:
scrapped;

In [8]:
type(scrapped)

bs4.element.ResultSet

In [9]:
type(scrapped[0].text)

str

In [10]:
# czyszczenie danych z tagów
clean_data = [d.text.lstrip('<td>').rstrip('</td>') for d in scrapped]
clean_data

['KOD POCZTOWY',
 'MIEJSCOWOŚĆ',
 '50-001',
 'Wrocław',
 '50-002',
 'Wrocław',
 '50-003',
 'Wrocław',
 '50-004',
 'Wrocław',
 '50-005',
 'Wrocław',
 '50-006',
 'Wrocław',
 '50-007',
 'Wrocław',
 '50-008',
 'Wrocław',
 '50-009',
 'Wrocław',
 '50-010',
 'Wrocław',
 '50-011',
 'Wrocław',
 '50-012',
 'Wrocław',
 '50-013',
 'Wrocław',
 '50-013',
 'Wrocław',
 '50-014',
 'Wrocław',
 '50-015',
 'Wrocław',
 '50-016',
 'Wrocław',
 '50-017',
 'Wrocław',
 '50-018',
 'Wrocław',
 '50-018',
 'Wrocław',
 '50-019',
 'Wrocław',
 '50-020',
 'Wrocław',
 '50-021',
 'Wrocław',
 '50-022',
 'Wrocław',
 '50-023',
 'Wrocław',
 '50-023',
 'Wrocław',
 '50-024',
 'Wrocław',
 '50-024',
 'Wrocław',
 '50-024',
 'Wrocław',
 '50-026',
 'Wrocław',
 '50-026',
 'Wrocław',
 '50-027',
 'Wrocław',
 '50-027',
 'Wrocław',
 '50-027',
 'Wrocław',
 '50-028',
 'Wrocław',
 '50-028',
 'Wrocław',
 '50-029',
 'Wrocław',
 '50-029',
 'Wrocław',
 '50-030',
 'Wrocław',
 '50-031',
 'Wrocław',
 '50-032',
 'Wrocław',
 '50-032',
 'Wrocław',
 

In [11]:
postal = []
city = []
for idx, element in enumerate(clean_data[2:]):
    if idx % 2 == 0:
        postal.append(element)
    else:
        city.append(element)

In [13]:
df = pd.DataFrame({'postal_code': postal, 'city': city})
df

,postal_code,city
0,50-001,Wrocław
1,50-002,Wrocław
2,50-003,Wrocław
3,50-004,Wrocław
4,50-005,Wrocław
...,...,...
2955,59-930,Pieńsk
2956,59-940,Węgliniec
2957,59-950,Ruszów
2958,59-970,Zawidów


In [14]:
#  dane województwa dolnośląskiego bez powiatu Wrocław
df_dolnoslaskie =  df[df['city'] != 'Wrocław']
df_dolnoslaskie

,postal_code,city
2601,55-002,Kamieniec Wrocławski
2602,55-003,Czernica
2603,55-010,Święta Katarzyna
2604,55-011,Siechnice
2605,55-015,Radwanice
...,...,...
2955,59-930,Pieńsk
2956,59-940,Węgliniec
2957,59-950,Ruszów
2958,59-970,Zawidów


In [15]:
# dane z kodami pocztowymi tylko powiatu Wrocław
df_wro = df[df['city'] == 'Wrocław']
df_wro

,postal_code,city
0,50-001,Wrocław
1,50-002,Wrocław
2,50-003,Wrocław
3,50-004,Wrocław
4,50-005,Wrocław
...,...,...
2596,54-703,Wrocław
2597,54-704,Wrocław
2598,54-704,Wrocław
2599,54-705,Wrocław


In [16]:
# kody pocztowe wszystkich miast w Polsce
df_all = pd.read_csv('kody.csv', 'UTF-8', delimiter= ';')
# kody pocztowe bez województwa dolnośląskiego
df_rest = df_all[['KOD POCZTOWY', 'MIEJSCOWOŚĆ']]
df_rest[df_rest['KOD POCZTOWY'].str[0] != "5"]
df_rest

,KOD POCZTOWY,MIEJSCOWOŚĆ
0,00-001,Warszawa
1,00-001,Warszawa
2,00-002,Warszawa
3,00-003,Warszawa
4,00-004,Warszawa
...,...,...
43779,99-418,Bełchów
43780,99-420,Łyszkowice
43781,99-423,Bielawy
43782,99-434,Domaniewice


In [17]:
random.seed(123)

# generowanie danych kod pocztowy i uwzględnieniem prawdopodobieństwa wystąpienia
def generate_postal_code(N):
    arr = np.random.choice((1,2,3), (N,), p=(0.6, 0.3, 0.1))
    postal = []
    for i in arr:
        if i == 1:
            kod_pocztowy = random.choice(df_wro['postal_code'])
        elif i == 2:
            kod_pocztowy = random.choice(df_dolnoslaskie['postal_code'].values)
        else:
            kod_pocztowy = random.choice(df_rest['KOD POCZTOWY'].values)
        postal.append(kod_pocztowy)
    return postal

postals = generate_postal_code(250)
postals = sorted(postals)

In [18]:
# wygenerowane dane
temp = pd.DataFrame({'KOD POCZTOWY': postals})
temp = temp.merge(df_all[['KOD POCZTOWY', 'MIEJSCOWOŚĆ']].drop_duplicates(), how='left', on = 'KOD POCZTOWY')
temp

,KOD POCZTOWY,MIEJSCOWOŚĆ
0,00-793,Warszawa
1,02-269,Warszawa
2,02-521,Warszawa
3,02-840,Warszawa
4,03-287,Warszawa
...,...,...
245,86-260,Unisław
246,86-308,Grudziądz
247,90-355,Łódź
248,91-364,Łódź


In [19]:
temp['ADRES'] = [fake.street_name() + " " + fake.building_number() for _ in range(len(temp))]
temp = temp.sort_values('KOD POCZTOWY')
temp['address_id'] = [i for i in range(1, 251)]

In [21]:
temp.columns = ['postal_code', 'city', 'address', 'address_id']
temp[['address_id', 'address', 'city', 'postal_code']]

,address_id,address,city,postal_code
0,1,Czereśniowa 30/97,Warszawa,00-793
1,2,Sucharskiego 61,Warszawa,02-269
2,3,Krasińskiego 43/15,Warszawa,02-521
3,4,Kochanowskiego 07,Warszawa,02-840
4,5,Wiosenna 225,Warszawa,03-287
...,...,...,...,...
245,246,Witosa 72,Unisław,86-260
246,247,Środkowa 16/04,Grudziądz,86-308
247,248,Podgórna 400,Łódź,90-355
248,249,Kosynierów 85,Łódź,91-364


In [22]:
temp.to_csv('address.csv')